In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.models import load_model
from keras_preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.callbacks import ReduceLROnPlateau
from keras import optimizers
import matplotlib.pyplot as plt
from keras import initializers
from keras.layers.normalization import BatchNormalization
from sklearn.utils import class_weight
from keras.optimizers import SGD

In [ ]:
train= pd.read_csv('/kaggle/input/plant-pathology-2020-fgvc7/train.csv')
train['image_id']=train['image_id']+'.jpg'

test=pd.read_csv('/kaggle/input/plant-pathology-2020-fgvc7/test.csv')
test['image_id']=test['image_id']+'.jpg'

train, val = train_test_split(train, test_size = 0.3)

In [ ]:
# CNN model
classifier = Sequential()
classifier.add(Convolution2D(128, (3,3), input_shape= (256, 256, 3),kernel_initializer='glorot_normal'))
classifier.add(BatchNormalization())
classifier.add(LeakyReLU(alpha=0.1))
classifier.add(MaxPooling2D(pool_size=[2,2]))

classifier.add(Convolution2D(64, (3,3), kernel_initializer='glorot_normal'))
classifier.add(BatchNormalization())
classifier.add(LeakyReLU(alpha=0.1))
classifier.add(MaxPooling2D(pool_size=[2,2]))

classifier.add(Convolution2D(64, (3,3), kernel_initializer='glorot_normal'))
classifier.add(BatchNormalization())
classifier.add(LeakyReLU(alpha=0.1))
classifier.add(MaxPooling2D(pool_size=[2,2]))

classifier.add(Convolution2D(64, (3,3), kernel_initializer='glorot_normal'))
classifier.add(BatchNormalization())
classifier.add(LeakyReLU(alpha=0.1))
classifier.add(MaxPooling2D(pool_size=[2,2]))

classifier.add(Convolution2D(32, (3,3),kernel_initializer='glorot_normal'))
classifier.add(BatchNormalization())
classifier.add(LeakyReLU(alpha=0.1))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

classifier.add(Convolution2D(32, (3,3),kernel_initializer='glorot_normal'))
classifier.add(BatchNormalization())
classifier.add(LeakyReLU(alpha=0.1))
#classifier.add(MaxPooling2D(pool_size=(2, 2)))

classifier.add(Convolution2D(8, (3,3),kernel_initializer='glorot_normal'))
classifier.add(BatchNormalization())
classifier.add(LeakyReLU(alpha=0.1))
classifier.add(MaxPooling2D(pool_size=(2, 2)))


classifier.add(Flatten())
classifier.add(Dense(output_dim = 128))
classifier.add(LeakyReLU(alpha=0.1))
classifier.add(Dropout(0.5))
classifier.add(Dense(output_dim = 4, activation= 'softmax' ))


#adam = optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
classifier.compile(optimizer= 'adagrad', loss = 'categorical_crossentropy', metrics=['accuracy'])

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

batch_s = 16
target_s = 256
train_generator = train_datagen.flow_from_dataframe(train, directory= '/kaggle/input/plant-pathology-2020-fgvc7/images/',
        target_size=(target_s, target_s),
        x_col="image_id",
        y_col=['healthy','multiple_diseases','rust','scab'],
        batch_size= batch_s,
        subset='training',
        class_mode='raw')

validation_generator = test_datagen.flow_from_dataframe(val, directory= '/kaggle/input/plant-pathology-2020-fgvc7/images/',
        target_size=(target_s, target_s),
        x_col="image_id",
        y_col= ['healthy','multiple_diseases','rust','scab'],
        class_mode = 'raw',
        batch_size=batch_s)

test_generator=test_datagen.flow_from_dataframe(test, directory='/kaggle/input/plant-pathology-2020-fgvc7/images/',
                                                      target_size=(target_s, target_s),
                                                      x_col="image_id",
                                                      y_col=None,
                                                      class_mode=None,
                                                      shuffle=False,
                                                      batch_size=batch_s)

In [ ]:
results = classifier.fit_generator(
        train_generator,
        epochs = 3,
        steps_per_epoch = 1200,
        validation_data = validation_generator,
        validation_steps = 800,
        callbacks=[ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3, min_lr=0.000001)])

In [ ]:
y_test=classifier.predict(test_generator, steps=test_generator.n/batch_s)
print(y_test.shape)

submission=pd.read_csv('/kaggle/input/plant-pathology-2020-fgvc7/sample_submission.csv')


for i,j in enumerate(['healthy','multiple_diseases','rust','scab']):
    submission[j]=y_test[:,i]
    
submission.to_csv('Highest_16_modified_30_validation.csv', index=False)

In [ ]:
#labelsmoothing
import pandas as pd

def LabelSmoothing(encodings , alpha):
    K = encodings.shape[1]
    y_ls = (1 - alpha) * encodings + alpha / K
    return y_ls
path1 = '/kaggle/input/plant-pathology-2020-fgvc7/sample_submission.csv'
path2 = '/kaggle/input/labelsmoothing/submission_16 (1).csv'
output_raw= pd.read_csv(path2)
new_output = LabelSmoothing(output_raw,0.01)